**1. Importing required Libraires**

In [113]:
import pandas as pd
import json

**2. Importing the Json files**

In [114]:
input_path = '/home/sbdesai2/chatbotai/saish/rlhf/data_prep/ScaleAI_results_Version3_set2.json'
json_path = open(input_path)
data = json.load(json_path)
data

[{'task_id': '63f51d5dc671ddcbe93f4152',
  'created_at': '2023-02-21T19:37:01.828Z',
  'completed_at': '2023-02-24T22:43:47.352Z',
  'type': 'textcollection',
  'status': 'completed',
  'instruction': ' ',
  'params': {'fields': [{'type': 'category',
     'field_id': '1st Choice Best Answer',
     'title': '1st Choice Best Answer',
     'choices': [{'label': 'Option 1', 'value': 'Option 1'},
      {'label': 'Option 2', 'value': 'Option 2'},
      {'label': 'Option 3', 'value': 'Option 3'},
      {'label': 'Option 4', 'value': 'Option 4'}],
     'description': 'Select the option corresponding to the best answer choice.'},
    {'type': 'category',
     'field_id': '2nd Choice',
     'title': '2nd Choice',
     'choices': [{'label': 'Option 1', 'value': 'Option 1'},
      {'label': 'Option 2', 'value': 'Option 2'},
      {'label': 'Option 3', 'value': 'Option 3'},
      {'label': 'Option 4', 'value': 'Option 4'}],
     'min_choices': 1,
     'max_choices': 1,
     'description': 'Select y

**3. Mapping question to the provided options**

In [115]:
qa_dict = {}
for i in range(len(data)):
    task_id = data[i]['task_id']
    q = (data[i]['params']['attachments'][0]['content'].split("?")[0] + "?").strip(" ")
    a1 = data[i]['params']['attachments'][0]['content'].split("\n")[-1]
    a2 = data[i]['params']['attachments'][1]['content'].split("\n")[-1]
    a3 = data[i]['params']['attachments'][2]['content'].split("\n")[-1]
    
    qa_dict[q] = {1 : a1, 
                  2 : a2, 
                  3 : a3}

print("Sample dictioanry entry")    
print("The question is: ", q)
print("The answer")
qa_dict['How can essential hazards be detected in a clocked synchronous design based on flip-flops?']

Sample dictioanry entry
The question is:  How does the bit-sliced adder circuit mimic a human's approach to adding two binary numbers?
The answer


{1: 'Essential hazards in a clocked synchronous design based on flip-flops can be detected by examining the sequential feedback state table. Each row contains one state, and each column contains the input bit that changes at a time. By analyzing this table, essential hazards related to clock skew can be identified.',
 2: 'Hazards in clocked synchronous designs can be detected by observing glitches in the output when the input shifts from one value to another. This is known as a static-1 hazard, which occurs when one of the AND gates driving the OR gate output goes low before the other rises.',
 3: 'Essential hazards in a clocked synchronous design based on flip-flops can be detected by examining the state machine and determining if the next state of the machine depends on the current state. If so, an essential hazard exists, as a second rising clock edge moves the system into a second new state.'}

**4. Generating  Binary Comparisons**

Generating binary pairs from the user selected and generated responses

Follwing are the cases of comparisons - 

1) If the question is bad -> ['Quest is Bad', Option 1], ['Quest is Bad', Option 2],['Quest is Bad', Option 3]


2) All other the pairs are grouped based on their selection priority and unselected options are given last preference.

3) If a user generated response is provided, then it is given higher preference over selected options


In [116]:
def ordered_pairs(test_list):
    res = []
    for idx, a in enumerate(test_list):
        for b in test_list[idx + 1:]:
            res.append([a,b])
    return res

In [117]:
qa_human_response = {}
for i in range(len(data)):
    task_id = data[i]['task_id']
    q = (data[i]['params']['attachments'][0]['content'].split("?")[0] + "?").strip(" ")
    c1 = data[i]['response']['annotations']['1st Choice Best Answer']
    c2 = data[i]['response']['annotations']['2nd Choice']
    c3 = data[i]['response']['annotations']['3rd Choice']
    userresponse = data[i]['response']['annotations']['labeler_generated_answer']
    userquest = data[i]['response']['annotations']['labeler_generated_question']
    badresponse = data[i]['response']['annotations']['Bad question']

    final_response_order = []
    selected_response = []
    unselected_response = []

    # Bad Question
    if len(badresponse) > 0:
        final_response_order = [badresponse + [1]] + [badresponse + [2]] + [badresponse + [3]]
    else:

        # selected response
        if len(c1) > 0:
            selected_response.append(int(c1[0].split(" ")[-1]))
        if len(c2) > 0:
            selected_response.append(int(c2[0].split(" ")[-1]))
        if len(c3) > 0:
            selected_response.append(int(c3[0].split(" ")[-1]))

        # unselected response
        if 1 not in selected_response:
            unselected_response.append(1)
        if 2 not in selected_response:
            unselected_response.append(2)
        if 3 not in selected_response:
            unselected_response.append(3)
        
        if len(unselected_response) != 0 and len(unselected_response) < 3:
            # final response order
            for i in range(len(unselected_response)):
                final_response_order.append(selected_response + [unselected_response[i]])
            
        else:
            final_response_order = [selected_response[:]]

        # user generated answer
        if userresponse != "":
            final_response_order = [[userresponse] + ele for ele in final_response_order]
            

    orderred_list = []
    for ele in final_response_order:
        orderred_list += ordered_pairs(ele)
    # print(selected_response)
    # print(unselected_response)
    # # print(final_response_order)
    # print(orderred_list)
    # print("**********")


    
    qa_human_response[q] = orderred_list

In [118]:
entry = []
l = []
for q in qa_human_response:
    for option in qa_human_response[q]:
        if isinstance(option[0], int):
            accept = qa_dict[q][option[0]]
        else:
            accept = option[0]

        if isinstance(option[1], int):
            reject = qa_dict[q][option[1]]
        else:
            reject = option[1]
        l.append([q,accept,reject])

data = pd.DataFrame(l, columns = ['Question', 'Chosen', 'Rejected'])   

In [119]:
data

,Question,Chosen,Rejected
0,How can essential hazards be detected in a clo...,Hazards in clocked synchronous designs can be ...,Essential hazards in a clocked synchronous des...
1,How can essential hazards be detected in a clo...,Hazards in clocked synchronous designs can be ...,Essential hazards in a clocked synchronous des...
2,How can essential hazards be detected in a clo...,Essential hazards in a clocked synchronous des...,Essential hazards in a clocked synchronous des...
3,What is Pareto optimality and how can it be us...,Question is bad,Pareto optimality is a concept used to identif...
4,What is Pareto optimality and how can it be us...,Question is bad,Pareto optimality is a concept used in multi-m...
...,...,...,...
297,What is the symbol used to represent an adder ...,The symbol used to represent an adder in logic...,The symbol used to represent an adder in logic...
298,What is the symbol used to represent an adder ...,The symbol used to represent an adder in logic...,The symbol used to represent an adder in logic...
299,How does the bit-sliced adder circuit mimic a ...,The bit-sliced adder circuit replicates the wa...,The bit-sliced adder circuit mimics the human ...
300,How does the bit-sliced adder circuit mimic a ...,The bit-sliced adder circuit replicates the wa...,The bit-sliced adder circuit mimics a human's ...
